In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_raw = root / 'data' / 'raw_data'
data_out = root / 'data' / 'processed_data'

print("Пути инициализированы:")
print(f"Raw data: {data_raw}")
print(f"Processed data: {data_out}")

Пути инициализированы:
Raw data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/raw_data
Processed data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data


In [2]:
# Загрузка данных
print("\nЗагружаем сырые данные...")
sessions_raw = pd.read_pickle(data_raw / 'ga_sessions.pkl')
hits_raw = pd.read_pickle(data_raw / 'ga_hits.pkl')
print("\nДанные успешно загружены!")


Загружаем сырые данные...

Данные успешно загружены!


In [3]:
def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")

    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())

    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")

    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")

    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")

    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return

    # Вывод типа данных
    print(f"Тип данных: {df[col].dtype}")

    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")

    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())

    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-5 значений:")
        print(df[col].value_counts(dropna=False).head(10))


generate_summary(sessions_raw, "Сырые данные сессий")
for col in sessions_raw.columns:
    analyze_column(sessions_raw, col, max_display=100)


================================================== СЫРЫЕ ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,860,042
Количество признаков: 18

Типы данных:
        count
object     17
int64       1

Пропущенные значения:
                 count     %
device_model   1843704 99.12
utm_keyword    1082061 58.17
device_os      1070138 57.53
utm_adcontent   335615 18.04
utm_campaign    219603 11.81
device_brand    118678  6.38
utm_source          97  0.01

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: object
Пропуски: 0 (0.0%)
Уникальных значений: 1860042

Слишком много значений (> 100). Примеры:
['8782573033728291727.1632416655.1632416655'
 '3377139778678328680.1626770792.1626770792'
 '4798853384778299931.1625570472.1625570472'
 '6753884663556557792.1632143862.1632143862'
 '7521592866376990851.1635991683.1635991683'
 '5365350113606796115.1629651803.1629651803'
 '5232509854625252375.1621836719.1621836719'
 '5798859708469419438.1638210990.1638210990'
 '5306689016481648634.1624969216.1624969216'
 '2414180090561573050.1639854316.1639854339']

Топ-5 значений:
session_id
9055434745589932991.1637753792.1637753792    1
6294640354816492625.1638728787.1638728809    1
6294832296889291185.1623245519.1623245519    1
6294832296889291185.1623161065.1623161065    1
6294832296889291185.1623156608.1623156608    1
6294832296889291185.1623069105.1623069105    1
6294825034110946181.1634421637.163

In [5]:
# Копируем данные
sessions = sessions_raw.copy()

# Объединяем 'visit_date' и 'visit_time' в 'visit_datetime'
sessions['visit_datetime'] = pd.to_datetime(
    sessions['visit_date'].astype(str) + ' ' + sessions['visit_time'].astype(str),
    errors='coerce'
)

# Удаляем старые столбцы 'visit_date' и 'visit_time'
sessions.drop(columns=['visit_date', 'visit_time'], inplace=True)

# Вставляем 'visit_datetime' после 'client_id'
sessions.insert(sessions.columns.get_loc('client_id') + 1, 'visit_datetime', sessions.pop('visit_datetime'))

# Извлекаем дополнительные данные из 'visit_datetime'
sessions['visit_hour'] = sessions['visit_datetime'].dt.hour
sessions['visit_weekday'] = sessions['visit_datetime'].dt.weekday
sessions['is_weekend'] = sessions['visit_weekday'] >= 5

# Добавляем новые столбцы после 'visit_datetime'
for col in ['is_weekend', 'visit_weekday', 'visit_hour']:
    sessions.insert(sessions.columns.get_loc('visit_datetime') + 1, col, sessions.pop(col))

# Удаляем столбец 'device_model' (99% пропусков)
sessions.drop(columns=['device_model'], inplace=True)

# Выделяем два отдельных признака высоту и ширину экрана
resolution = (
        sessions['device_screen_resolution']
        .str.extract(r'(?P<device_screen_width>\d+)x(?P<device_screen_height>\d+)')
    )

sessions['device_screen_width'] = resolution['device_screen_width'].astype('Int32')
sessions['device_screen_height'] = resolution['device_screen_height'].astype('Int32')

# Удаляем старый столбец 'device_screen_resolution'
sessions.drop(columns=['device_screen_resolution'], inplace=True)

# Добавляем новые столбцы после 'device_brand'
for col in ['device_screen_width', 'device_screen_height']:
    sessions.insert(sessions.columns.get_loc('device_brand') + 1, col, sessions.pop(col))
#================================================================
# Приводим столбцы к правильным типам и заменяем значения
#sessions = sessions.convert_dtypes()
#================================================================
# Заменяем значения в нескольких столбцах на pd.NA

bad = ['(none)', 'nan', '(not set)', 'NaN', 'None', '']

replace_dict = {
    'utm_source':bad,
    'utm_medium': bad,
    'utm_campaign':bad,
    'utm_adcontent': bad,
    'utm_keyword': bad,
    'device_category':bad,
    'device_os': bad,
    'device_browser': bad,
    'device_brand': bad,
    'geo_country': bad,
    'geo_city': bad,
    'device_screen_width':[0],
    'device_screen_height': [0]

}

for col, values in replace_dict.items():
    for value in values:
        sessions[col] = sessions[col].replace(value, pd.NA)
#==========================================================================
# Удаляем строки с пропущенным значением (количество <100 строк)
sessions.dropna(subset=['utm_source'], inplace=True)
sessions.dropna(subset=['device_browser'], inplace=True)
sessions.dropna(subset=['device_screen_width'], inplace=True)
sessions.dropna(subset=['device_screen_height'], inplace=True)
#==========================================================================
# Заполняем пропуски информации о девайсах исходя из его категории

def fill_missing(df, categories, fields, mask_category):

    for category in categories:
        for field in fields:
            placeholder = f'noname_{category}_{field.split("_")[1]}'
            mask = (df[mask_category] == category) & (df[field].isna())
            df.loc[mask, field] = placeholder

    return df

device_categories = ['mobile', 'desktop', 'tablet']
device_fields = ['device_os', 'device_brand', 'device_browser']
device_mask_category = 'device_category'

sessions = fill_missing(sessions, device_categories, device_fields, device_mask_category)

#===========================================================================
# Заполняем пропуски геопозиции
#===========================================================================
def mark_abroad(df: pd.DataFrame, country_col: str = 'geo_country') -> pd.DataFrame:
    """
    Если значение в столбце country_col не Россия — подставляется 'abroad'.
    Пропуски остаются как есть.
    """
    df[country_col] = df[country_col].apply(
        lambda x: 'russia' if isinstance(x, str) and x.strip().lower() == 'russia'
        else ('abroad' if pd.notna(x) else x)
    )
    return df

sessions = mark_abroad(sessions, country_col='geo_country')

sessions['geo_country'] = sessions['geo_country'].fillna('missing')

geo_categories = ['russia', 'abroad', 'missing']
geo_fields = ['geo_city']
geo_mask_category = 'geo_country'

sessions = fill_missing(sessions, geo_categories, geo_fields, geo_mask_category)

#===========================================================================
# Заполняем пропуски в utm_*

sessions['utm_medium'] = sessions['utm_medium'].fillna('missing')
sessions['utm_campaign'] = sessions['utm_campaign'].fillna('missing')
sessions['utm_adcontent'] = sessions['utm_adcontent'].fillna('missing')
sessions['utm_keyword'] = sessions['utm_keyword'].fillna('missing')

# Приводим столбцы к правильным типам и заменяем значения
sessions = sessions.convert_dtypes()

# Генерация отчета
generate_summary(sessions, "Данные сессий")
for col in sessions.columns:
    analyze_column(sessions, col, max_display=100)



================================================== ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,859,909
Количество признаков: 20

Типы данных:
                count
string[python]     13
Int32               4
datetime64[ns]      1
boolean             1
Int64               1

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,True,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28 02:42:06,2,1,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1859909

Слишком много значений (> 100). Примеры:
<StringArray>
['3742168130055334387.1639737991.1639737991',
 '4414972965969603293.1636969184.1636969184',
 '7084552821274296393.1632996609.1632996609',
  '877962660635291148.1631227323.1631227323',
 '1073097309958251563.1635593193.1635593193',
  '778497789364439680.1635682943.1635682943',
 '2676609378396114193.1640467728.1640467728',
 '5794457083522625231.1638877508.1638877508',
 '1730605233294025408.1626088133.1626088133',
 '4188425235078188321.1629359394.1629359394']
Length: 10, dtype: string

Топ-5 значений:
session_id
9055434745589932991.1637753792.1637753792    1
9055645813163970503.1633964004.1633964004    1
9055447046360770272.1622255328.1622255328    1
9055447046360770272.1622255345.1622255345    1
9055447192389856083.1622453074.1622453074    1
9055455318486370642.1640843788.1

In [6]:
generate_summary(hits_raw, "Сырые данные событий")
for col in hits_raw.columns:
    analyze_column(hits_raw, col, max_display=100)


================================================== СЫРЫЕ ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 11

Типы данных:
         count
object       9
float64      1
int64        1

Пропущенные значения:
                count      %
event_value  15726470 100.00
hit_time      9160322  58.25
hit_referer   6274804  39.90
event_label   3760184  23.91

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.164025...,2021-12-23,597864.00,30,event,NaN,sberauto.com/cars?utm_sour...,quiz,quiz_show,NaN,None
1,7750352294969115059.164027...,2021-12-23,597331.00,41,event,NaN,sberauto.com/cars/fiat?cit...,quiz,quiz_show,NaN,None
2,885342191847998240.1640235...,2021-12-23,796252.00,49,event,NaN,sberauto.com/cars/all/volk...,quiz,quiz_show,NaN,None



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: object
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
['2349301066205103009.1637483544.1637483544'
 '2346269038334200929.1634406970.1634406970'
 '8707723543437043929.1625221337.1625221337'
 '4726166187204433608.1640519334.1640519334'
 '1814790049758540631.1640330070.1640330070'
 '4352232951296158142.1640098240.1640098240'
 '7934652064631495946.1622030919.1622030919'
 '3658409752318731622.1640523110.1640523110'
 '4991910463941736674.1640855778.1640855778'
 '999871845591953061.1634479785.1634479785']

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1629319807.1629319807    496
686125592720823356.1634

In [7]:
# Копируем данные
hits = hits_raw.copy()

# Удаляем столбецы 'event_value', 'hit_type' (неинформативны)
hits.drop(columns=['event_value', 'hit_type'], inplace=True)
#==============================================================
# Приводим hit_date и hit_time
hits['hit_date_dt'] = pd.to_datetime(hits['hit_date'], format='%Y-%m-%d', errors='coerce')

hits['hit_time_td'] = pd.to_timedelta(hits['hit_time'], unit='ms')

# Удаляем старые столбцы 'hit_date' и 'hit_time'
hits.drop(columns=['hit_date', 'hit_time'], inplace=True)
#=============================================================
# Добавляем новые столбцы после 'device_brand'
for col in ['hit_time_td', 'hit_date_dt']:
    hits.insert(hits.columns.get_loc('session_id') + 1, col, hits.pop(col))
#=============================================================
# Приводим столбцы к правильным типам и заменяем значения
hits = hits.convert_dtypes()
#================================================================
# Заполняем пропуски, когда hit_number = 1
mask = (hits['hit_number'] == 1) & (hits['hit_time_td'].isna())
hits.loc[mask, 'hit_time_td'] = pd.Timedelta(0)

# Заполняем пропуски hit_time_dt с учетом hit_number
# Вычисляем моду (самое частое) для каждой группы hit_number
mode_per_hit = (
    hits
    .groupby('hit_number')['hit_time_td']
    .agg(lambda x: x.mode(dropna=True).iloc[0] if not x.mode(dropna=True).empty else pd.Timedelta(0))
)

# Заполняем пропуски в исходном столбце соответствующими модами
hits['hit_time_td'] = hits['hit_time_td'].fillna(
    hits['hit_number'].map(mode_per_hit)
)
#=================================================================
# Заполняем пропуски в категориальных столбцах
hits['hit_referer'] = hits['hit_referer'].fillna('missing')
hits['event_label'] = hits['event_label'].fillna('missing')
#=================================================================

# Объявляем целевые действия
target_actions = [
    # Отправка форм и заявок
    'sub_submit_success', 'sub_submit_error',          # sub_submit
    'form_request_call_sent',                          # phone
    'sub_callback_submit_click',                       # sub_button_click
    'sub_car_request_submit_click',                    # sub_button_click
    'sub_custom_question_submit_click',                # sub_button_click
    'sub_car_claim_submit_click',                      # sub_button_click

    # Обратный звонок и контакт
    'click_on_request_call', 'click_on_phone',          # phone
    'phone_entered_on_form_request_call',               # phone
    'showed_form_request_call',                         # phone

    # Чат и онлайн-диалог
    'start_chat', 'chat requested',                     # chat, jivosite
    'user_message', 'client initiate chat',             # chat, jivosite
    'callback requested',                               # jivosite
    'offline form shown',                               # jivosite

    # Аутентификация (сильный признак санкционированного взаимодействия)
    'phone_auth_success', 'sber_id_auth_success',       # auth

    # Кредит и рассчет
    'calculate',                                        # credit_landing_pos
    'request', 'request_success',                       # credit_landing_pos

    # Клики по покупке/оформлению подписки
    'click_on_subscription',                            # header, main_banners
    'click_auto_subscription',                          # main_services
    'click_free_car_selection',                         # main_services, chat
    'click_buy_auto',                                   # main_advantages, main_services
    'click_auto',                                       # main_services
    'click_car_selection',                              # main_services
    'click_car_buyback',                                # header, main_services
    'click_setelem_credit',                             # main_services
    'click_pos_credit',                                 # main_services

    # Публикация объявления (для частных продавцов)
    'success_ad_creation',                              # add_ad_publish_success

    # Запросы по VIN, цене, городу — сильный признак готовности
    'forward_to_price', 'sap_search_form_cost_to',      # add_ad_city, sap_search_form
    'sap_search_form_cost_from',                       # sap_search__form

    # Переходы из листинга и карточки
    'go_to_car_card',                                   # listing_ads, greenday_listing_ads
    'sub_view_cars_click',                              # sub_button_click, greenday_gtm.triggergroup

    # Клики из “подписаться”
    'sub_open_dialog_click',                            # sub_button_click, greenday_sub_button_click

    # Дополнительные целевые события
    'request',                                          # credit_landing_pos (повтор)
    'order_call',                                       # если встречается в схеме
]

hits['is_target'] = hits['event_action'].isin(target_actions).astype('Int32')

#============================================================
generate_summary(hits, "Данные событий")
for col in hits.columns:
    analyze_column(hits, col, max_display=100)



================================================== ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 10

Типы данных:
                 count
string[python]       6
datetime64[ns]       1
timedelta64[ns]      1
Int64                1
Int32                1

Пропущенных значений нет

Дубликаты: 94 (0.00%)

Первые 3 записей:


,session_id,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,5639623078712724064.164025...,2021-12-23,0 days 00:09:57.864000,30,missing,sberauto.com/cars?utm_sour...,quiz,quiz_show,missing,0
1,7750352294969115059.164027...,2021-12-23,0 days 00:09:57.331000,41,missing,sberauto.com/cars/fiat?cit...,quiz,quiz_show,missing,0
2,885342191847998240.1640235...,2021-12-23,0 days 00:13:16.252000,49,missing,sberauto.com/cars/all/volk...,quiz,quiz_show,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
<StringArray>
['3447176844017377366.1637589077.1637589077',
 '7646265910034029420.1640094573.1640094573',
  '428517491566822420.1625408532.1625408532',
  '348937571546817059.1624629869.1624629869',
 '1897588089987741238.1639659059.1639659059',
 '5520402740846458029.1632378814.1632378814',
 '6730500778533164871.1626707784.1626707784',
 '3117289979493821241.1637690172.1637690172',
 '8289748802347255066.1629537564.1629537564',
 '4791016113151536743.1636953755.1636953755']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.16

In [8]:
# Копируем обработанные сессии и события
hits_p = hits.copy()
sessions_p = sessions.copy()
#============================================================
# Соеденяем в один большой датасет
full_raw = sessions_p.merge(
    hits_p,
    how='inner',
    on='session_id',
    suffixes=('_sess','_hit')
)
#============================================================
generate_summary(full_raw, "Общие данные")
for col in full_raw.columns:
    analyze_column(full_raw, col, max_display=100)


================================================== ОБЩИЕ ДАННЫЕ ==================================================
Общее количество записей: 15,684,368
Количество признаков: 29

Типы данных:
                 count
string[python]      18
Int32                5
datetime64[ns]       2
Int64                2
boolean              1
timedelta64[ns]      1

Пропущенных значений нет

Дубликаты: 94 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:03.665000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0
1,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:46.592000,4,missing,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,1
2,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,True,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,2021-11-14,0 days 00:00:00.921000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1732170

Слишком много значений (> 100). Примеры:
<StringArray>
['3111677711361930377.1631220875.1631220875',
 '6981064991575159790.1623241713.1623241713',
 '5993332619288281553.1634763222.1634763222',
 '1221060183127190747.1624833972.1624833972',
 '7629062470064466248.1635051850.1635051850',
 '8884655593071428362.1622524618.1622524618',
 '1349453243991726016.1629493184.1629493213',
 '8241363423078308706.1635441567.1635441567',
 '8211198407464244160.1630065606.1630065606',
 '3297934478597850957.1633060688.1633060688']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.16

In [9]:
# Копируем датасет
full = full_raw.copy()
#========================================================
#Удалим дубликаты
full.drop_duplicates(inplace=True, keep='first')
#========================================================
# Приводим типы
full['visit_number'] = full['visit_number'].astype('Int32')
full['hit_number'] = full['hit_number'].astype('Int32')
full['is_weekend'] = full['is_weekend'].astype('Int32')
#full['aspect_ratio'] = full['aspect_ratio'].astype('Float32')
#=============================================================
generate_summary(full, "Общие данные")
for col in full.columns:
    analyze_column(full, col, max_display=100)


================================================== ОБЩИЕ ДАННЫЕ ==================================================
Общее количество записей: 15,684,274
Количество признаков: 29

Типы данных:
                 count
string[python]      18
Int32                8
datetime64[ns]       2
timedelta64[ns]      1

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:03.665000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0
1,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:46.592000,4,missing,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,1
2,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,1,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,2021-11-14,0 days 00:00:00.921000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1732170

Слишком много значений (> 100). Примеры:
<StringArray>
['1974363761421358834.1626117874.1626117874',
 '7490836781665337194.1637334905.1637334905',
 '4185653731339389862.1631964073.1631964073',
  '705095866269043244.1636829615.1636829615',
 '6030404672443523758.1624908462.1624908462',
 '2307111525144003378.1634407981.1634407981',
 '2321779293714538992.1622565360.1622565360',
 '8260922339061768862.1625137822.1625137822',
  '573156622716776553.1639067754.1639067754',
 '7946238756819655701.1621974037.1621975334']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
686125592720823356.163

In [10]:
print("\nСохраняем обработанные данные...")
data_out.mkdir(parents=True, exist_ok=True)
full.to_pickle(data_out / 'data_processed.pkl')
print(f"Данные сохранены в: {data_out}")


Сохраняем обработанные данные...
Данные сохранены в: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data
